In [34]:
import os
import numpy as np

base_path = '/mnt/disk1/heonseok/MPMLD/output'

In [64]:
dataset = 'location'

beta_list = [
    # 0.00001,
    0.0001,
    # 0.001,
    # 0.01,
    # 0.1,
    # 1.0,
]

z_dim_list = [
    # '16',
    '64',
]

setsize_list = [
    # '500',
    # '1000',
    '2000',
]

lr_list = [
    '0.001',
    # '0.01',
    # '0.1',
]

ref_list = [
    '0.1',
]

arc_list = [
    # 'A',
    # 'B',
    # 'C',
    'D',
]

dis_type_list = [
    'base',
    # 'type5',
]

cw_list = [
    '0.01',
    '0.1',
    '1.0',
]

mw_list = [
    # '0.01',
    # '0.1',
    '1.0',
]

recon_model_list = []
for beta in beta_list:
    for z_dim in z_dim_list:
        for setsize in setsize_list:
            for lr in lr_list:
                for ref in ref_list:
                    for arc in arc_list:
                        for dis_type in dis_type_list:
                            if 'base' in dis_type:
                                recon_model = 'VAE{}_z{}_setsize{}_lr{}_ref{}_arc{}_{}'.format(
                                    beta, z_dim, setsize, lr, ref, arc, dis_type,
                                )
                                recon_model_list.append(recon_model)
                                continue
                            for cw in cw_list:
                                for mw in mw_list:
                                    recon_model = 'VAE{}_z{}_setsize{}_lr{}_ref{}_arc{}_{}_cw{}_mw{}'.format(
                                        beta, z_dim, setsize, lr, ref, arc, dis_type, cw, mw,
                                    )
                                    recon_model_list.append(recon_model)

for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

class_model_list = []
for idx in range(len(recon_model_list)):
    class_model_list.append(recon_model_list[idx] + '_FCNClassifierA')
    print('\''+class_model_list[idx]+'\',')

'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_base',
'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_base_FCNClassifierA',


In [59]:
for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

for recon_model in recon_model_list:
    recon_path = os.path.join(base_path, dataset, 'reconstructor', recon_model)

    # class_acc_full_list = []
    # class_acc_content_list = []
    # class_acc_style_list = []
    # membership_acc_full_list = []
    # membership_acc_content_list = []
    # membership_acc_style_list = []

    for repeat in range(5):
        try:
            recon_repeat_path = os.path.join(recon_path, 'repeat{}'.format(repeat))
            recon_mse_path = os.path.join(recon_repeat_path, 'mse.npy')
            recon_mse = np.load(recon_mse_path, allow_pickle=True)
            # print(recon_mse)

            if repeat == 0:
                mse_list = recon_mse
            else:
                mse_list += recon_mse
            # class_acc_full_list.append(recon_acc['class_acc_full'])
            # class_acc_content_list.append(recon_acc['class_acc_content'])
            # class_acc_style_list.append(recon_acc['class_acc_style'])
            # membership_acc_full_list.append(recon_acc['membership_acc_full'])
            # membership_acc_content_list.append(recon_acc['membership_acc_content'])
            # membership_acc_style_list.append(recon_acc['membership_acc_style'])

        except FileNotFoundError:
            # print('File not found: ', recon_acc_path)
            continue

    mse_list = mse_list/5

    result_list = []
    for idx, mse in enumerate(mse_list):
        result_list.append('{:.4f}'.format(mse))
    print(','.join(result_list))

    # result_list = [
    #     *recon_model.split('_'),
    #     np.average(class_acc_full_list), np.average(class_acc_content_list), np.average(class_acc_style_list),
    #     np.average(membership_acc_full_list), np.average(membership_acc_content_list), np.average(membership_acc_style_list)
    # ]
    #
    # for idx, result in enumerate(result_list):
    #     if not type(result) is str:
    #         result_list[idx] = '{:.4f}'.format(result)
    # print(','.join(result_list))

'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_type5_cw0.01_mw1.0',
'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_type5_cw0.1_mw1.0',
'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_type5_cw1.0_mw1.0',
0.0086,0.0107,0.0106,0.0087,0.0108,0.0107,0.0087,0.0108,0.0107,0.0087,0.0108,0.0107
0.0089,0.0110,0.0109,0.0090,0.0110,0.0109,0.0090,0.0110,0.0109,0.0091,0.0111,0.0110
0.0076,0.0108,0.0107,0.0077,0.0108,0.0107,0.0077,0.0108,0.0107,0.0077,0.0109,0.0108


In [65]:
for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

for recon_model in recon_model_list:
    recon_path = os.path.join(base_path, dataset, 'reconstructor', recon_model)

    class_acc_full_list = []
    class_acc_content_list = []
    class_acc_style_list = []
    membership_acc_full_list = []
    membership_acc_content_list = []
    membership_acc_style_list = []

    for repeat in range(5):
        try:
            recon_repeat_path = os.path.join(recon_path, 'repeat{}'.format(repeat))
            recon_acc_path = os.path.join(recon_repeat_path, 'acc.npy')
            recon_acc = np.load(recon_acc_path, allow_pickle=True).item()

            class_acc_full_list.append(recon_acc['class_acc_full'])
            class_acc_content_list.append(recon_acc['class_acc_content'])
            class_acc_style_list.append(recon_acc['class_acc_style'])
            membership_acc_full_list.append(recon_acc['membership_acc_full'])
            membership_acc_content_list.append(recon_acc['membership_acc_content'])
            membership_acc_style_list.append(recon_acc['membership_acc_style'])

        except FileNotFoundError:
            # print('File not found: ', recon_acc_path)
            continue

    result_list = [
        *recon_model.split('_'),
        np.average(class_acc_full_list), np.average(class_acc_content_list), np.average(class_acc_style_list),
        np.average(membership_acc_full_list), np.average(membership_acc_content_list), np.average(membership_acc_style_list)
    ]

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_base',
VAE0.0001,z64,setsize2000,lr0.001,ref0.1,arcD,base,0.9515,0.6145,0.5985,0.9960,0.9517,0.9557


In [66]:
for class_model in class_model_list:
    print('\''+class_model+'\',')

for class_model in class_model_list:
    class_path = os.path.join(base_path, dataset, 'classifier', class_model)

    result_list = []
    for recon_type in ['base_z', 'content_z', 'style_z', 'full_z']:
        train_acc_list = []
        valid_acc_list = []
        test_acc_list = []
        for repeat in range(5):
            try:
                class_repeat_path = os.path.join(class_path, recon_type, 'repeat{}'.format(repeat))
                class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
                train_acc_list.append(class_acc['train'])
                valid_acc_list.append(class_acc['valid'])
                test_acc_list.append(class_acc['test'])

            except FileNotFoundError:
                # print('File not found: ', class_repeat_path)
                continue

        result_list.extend([np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_base_FCNClassifierA',
0.7520,0.6625,0.6100,0.8190,0.6800,0.6375,0.8500,0.6275,0.6375,0.7925,0.6625,0.6375


In [68]:
metric = 'acc'
# metric = 'auroc'

attack_type_list = ['stat', 'black']
for class_model in class_model_list:
    print('\''+class_model+'\',')

for class_model in class_model_list:
    attack_path = os.path.join(base_path, dataset, 'attacker', class_model)

    result_list = []
    for recon_type in ['base_z', 'content_z', 'style_z', 'full_z']:
        stat_acc_list = []
        black_acc_list = []
        # white_acc_list = []
        for repeat in range(5):
            for attack_type in attack_type_list:
                attack_repeat_path = os.path.join(attack_path, recon_type, 'repeat{}'.format(repeat))
                try:
                    attack_acc = np.load(os.path.join(attack_repeat_path, attack_type, '{}.npy'.format(metric)), allow_pickle=True)
                    if attack_type == 'stat':
                        stat_acc_list.append(attack_acc)
                    elif attack_type == 'black':
                        black_acc_list.append(attack_acc.item()['test'])
                    # elif attack_type == 'white':
                    #     white_acc_list.append(attack_acc.item()['test'])

                except FileNotFoundError:
                    # print('File not found: ', attack_repeat_path)
                    continue

        result_list.extend([np.average(stat_acc_list), np.average(black_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


'VAE0.0001_z64_setsize2000_lr0.001_ref0.1_arcD_base_FCNClassifierA',
0.5360,0.5533,0.5515,0.6050,0.5415,0.5717,0.5455,0.5683


In [40]:
# metric = 'acc'
metric = 'auroc'

classifier_with_raw_data = 'original_setsize2000_FCNClassifierA'
class_path = os.path.join(base_path, dataset, 'classifier', classifier_with_raw_data)
attack_path = os.path.join(base_path, dataset, 'attacker', classifier_with_raw_data)
train_acc_list = []
valid_acc_list = []
test_acc_list = []
stat_acc_list = []
black_acc_list = []
for repeat in range(5):
    try:
        class_repeat_path = os.path.join(class_path, 'repeat{}'.format(repeat))
        attack_repeat_path = os.path.join(attack_path, 'repeat{}'.format(repeat))

        class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
        train_acc_list.append(class_acc['train'])
        valid_acc_list.append(class_acc['valid'])
        test_acc_list.append(class_acc['test'])

        for attack_type in attack_type_list:
            attack_acc = np.load(os.path.join(attack_repeat_path, attack_type, '{}.npy'.format(metric)), allow_pickle=True)
            if attack_type == 'stat':
                stat_acc_list.append(attack_acc)
            elif attack_type == 'black':
                black_acc_list.append(attack_acc.item()['test'])

    except FileNotFoundError:
        # print('File not found: ', class_repeat_path)
        continue

print('{:.4f} {:.4f} {:.4f} {:.4f} {:.4f}'.format(
    np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list),
    np.average(stat_acc_list), np.average(black_acc_list),
))
# print(np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list))
# print(np.average(stat_acc_list), np.average(black_acc_list), np.average(test_acc_list))



0.9420 0.6850 0.6535 0.6066 0.7298
